<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>python-awips: Working with the Maps and Topo Databases</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

<div style="float:right; width:250 px"><img src="../../instructors/images/python-awips-maps-topo-preview.png" alt="Map of Boulder created with python-awips" style="height: 300px;"></div>


### Focuses

* Use the AWIPS Maps Database to access GIS objects which are returned as <a href="https://shapely.readthedocs.io/en/stable/manual.html">Shapely</a> geometries (*Polygon*, *Point*, *MultiLineString*, etc.) and can be easily plotted by Matplotlib, Cartopy, MetPy, and other packages. 
* Use **maps** and **topo** data types to obtain GIS data from the AWIPS databases.
* Note the geometry definition of `the_geom` for each data type, which can be **Point**, **MultiPolygon**, or **MultiLineString**.
* Step through how to plot several layers of data onto an image, including: county and state boundaries, CWA boundaries, interstates, cities, lakes, rivers, and topograpy.



### Objectives

1. [Define Map Data Request](#1.-Define-the-Map-Data-Request)
1. [Define Map Properties](#2.-Define-Map-Properties)
1. [Draw County and State Boundaries](#3.-Draw-County-and-State-Boundaries)
1. [Draw CWA Boundary](#4.-Draw-CWA-Boundary)
1. [Draw Interstates](#5.-Draw-Interstates)
1. [Draw Nearby Cities](#6.-Draw-Nearby-Cities)
1. [Draw Lakes](#7.-Draw-Lakes)
1. [Draw Major Rivers](#8.-Draw-Major-Rivers)
1. [Draw Topography](#9.-Draw-Topography)


---

### Imports

In [ ]:
from __future__ import print_function
from awips.dataaccess import DataAccessLayer
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature,NaturalEarthFeature
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import numpy.ma as ma

## 1. Define the Map Data Request

If you read through the [python-awips: How to Access Data](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/dataAccess/python-awips-HowToAccessData.ipynb) training, you will know that we need to set an EDEX url to access our server, and then we create a data request.  In this example we use *maps* as the data type to define our request.  We'll use Boulder as our location, so set the *Location Name* on the request to BDU.  Then add several *Identifiers* for various fields of interest.

In [ ]:
# Create EDEX data request
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest('maps')
request.addIdentifier('table', 'mapdata.county')

# Define a WFO ID for location
# tie this ID to the mapdata.county column "cwa" for filtering
request.setLocationNames('BOU')
request.addIdentifier('cwa', 'BOU')

# enable location filtering (inLocation)
# locationField is tied to the above cwa definition (BOU)
request.addIdentifier('geomField', 'the_geom')
request.addIdentifier('inLocation', 'true')
request.addIdentifier('locationField', 'cwa')

# take a look at the request
print(request)

<a href="#top">Top</a>

---

## 2. Define Map Properties

If more than one plot is drawn, then it's easiest to define common logic in a function.  Here, a new function called **make_map** is defined.  This function uses the [matplotlib.pyplot package (plt)](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.html) to create a figure and axis.  The coastlines (continental boundaries) are added, along with lat/lon grids.  

> Note: We only use this function once in this notebook, but it's in here as an example of how to write a function and use it, for future reference.

In [ ]:
# Standard map plot
def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(12,12),
            subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.coastlines(resolution='50m')
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

<a href="#top">Top</a>

---

## 3. Draw County and State Boundaries

Start by getting the [GeometryData](http://unidata.github.io/python-awips/api/PyGeometryData.html) back from EDEX for the map request.  We'll create a plot of the county boundaries for the WFO (in this case Boulder - BDU). Add in the 

In [ ]:
# Get response and create dict of county geometries
response = DataAccessLayer.getGeometryData(request, [])

# Populate an array of the counties for BDU
counties = np.array([])
for ob in response:
    counties = np.append(counties,ob.getGeometry())


# All WFO counties merged to a single Polygon
merged_counties = cascaded_union(counties)
envelope = merged_counties.buffer(2)
boundaries=[merged_counties]

# Get bounds of this merged Polygon to use as buffered map extent
bounds = merged_counties.bounds
bbox=[bounds[0]-1,bounds[2]+1,bounds[1]-1.5,bounds[3]+1.5]

# Create the map using our defined function
fig, ax = make_map(bbox=bbox)

# Plot state boundaries handled by Cartopy
states = NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', scale='50m', facecolor='none')
ax.add_feature(states, linestyle='-', edgecolor='black',linewidth=2)

# Plot CWA counties
for i, geom in enumerate(counties):
    cbounds = Polygon(geom)
    intersection = cbounds.intersection
    geoms = (intersection(geom) for geom in counties if cbounds.intersects(geom))
    shape_feature = ShapelyFeature(geoms,ccrs.PlateCarree(), facecolor='none', linestyle="-",edgecolor='#86989B')
    ax.add_feature(shape_feature)

<a href="#top">Top</a>

---

## 4. Draw CWA Boundary

Use the single polygon that encompases all of the counties in the CWA for Boulder to draw that on top of the previous figure.

In [ ]:
# use the merged polygon to draw the CWA boundary
geom = boundaries[0]
gbounds = Polygon(geom)
intersection = gbounds.intersection
geoms = (intersection(geom) for geom in boundaries if gbounds.intersects(geom))
shape_feature = ShapelyFeature(geoms,ccrs.PlateCarree(), facecolor='none', linestyle="-",linewidth=3.,edgecolor='#cc5000')
ax.add_feature(shape_feature)

# Display the plot
fig

<a href="#top">Top</a>

---

## 5. Draw Interstates

Using the previously-defined **envelope=merged_counties.buffer(2)** in **newDataRequest()** to request geometries which fall inside the buffered boundary. 

In [ ]:
# Create new request to get interstate polygons from the EDEX server
request = DataAccessLayer.newDataRequest('maps', envelope=envelope)
request.addIdentifier('table', 'mapdata.interstate')
request.addIdentifier('geomField', 'the_geom')
request.setParameters('name')
interstates = DataAccessLayer.getGeometryData(request, [])

# Plot interstates
for ob in interstates:
    shape_feature = ShapelyFeature(ob.getGeometry(),ccrs.PlateCarree(), facecolor='none', linestyle="-",edgecolor='orange')
    ax.add_feature(shape_feature)

# Display the plot
fig

<a href="#top">Top</a>

---

## 6. Draw Nearby Cities

Request the city table and filter by population and progressive disclosure level:

> **Warning**: the `prog_disc` field is not entirely understood and values appear to change significantly depending on WFO site.  

In [ ]:
# Create new request for local cities
request = DataAccessLayer.newDataRequest('maps', envelope=envelope)
request.addIdentifier('table', 'mapdata.city')
request.addIdentifier('geomField', 'the_geom')
request.setParameters('name','population','prog_disc')
cities = DataAccessLayer.getGeometryData(request, [])

citylist = []
cityname = []
# For BOU, progressive disclosure values above 50 and pop above 5000 looks good
for ob in cities:
    if ob.getString("population"):
        if ob.getNumber("prog_disc") > 50:
            if int(ob.getString("population")) > 5000:
                citylist.append(ob.getGeometry())
                cityname.append(ob.getString("name"))

# Plot city markers
ax.scatter([point.x for point in citylist], [point.y for point in citylist], transform=ccrs.PlateCarree(),marker="+",facecolor='black')
# Plot city names
for i, txt in enumerate(cityname):
    ax.annotate(txt, (citylist[i].x,citylist[i].y), xytext=(3,3), textcoords="offset points")

# Display the plot
fig

<a href="#top">Top</a>

---

## 7. Draw Lakes

Get lake geometry data from the map database from the EDEX server and add it to the plot.

In [ ]:
# Create a request for lakes
request = DataAccessLayer.newDataRequest('maps', envelope=envelope)
request.addIdentifier('table', 'mapdata.lake')
request.addIdentifier('geomField', 'the_geom')
request.setParameters('name')

# Get lake geometries
response = DataAccessLayer.getGeometryData(request, [])
lakes = np.array([])
for ob in response:
    lakes = np.append(lakes,ob.getGeometry())

# Plot lakes
for i, geom in enumerate(lakes):
    cbounds = Polygon(geom)
    intersection = cbounds.intersection
    geoms = (intersection(geom) for geom in lakes if cbounds.intersects(geom))
    shape_feature = ShapelyFeature(geoms,ccrs.PlateCarree(), facecolor='blue', linestyle="-",edgecolor='#20B2AA')
    ax.add_feature(shape_feature)

# Display the plot
fig

<a href="#top">Top</a>

---

## 8. Draw Major Rivers

In [ ]:
# Create a new request for major rivers
request = DataAccessLayer.newDataRequest('maps', envelope=envelope)
request.addIdentifier('table', 'mapdata.majorrivers')
request.addIdentifier('geomField', 'the_geom')
request.setParameters('pname')
rivers = DataAccessLayer.getGeometryData(request, [])

# Plot rivers
for ob in rivers:
    shape_feature = ShapelyFeature(ob.getGeometry(),ccrs.PlateCarree(), facecolor='none', linestyle=":", edgecolor='#20B2AA')
    ax.add_feature(shape_feature)

# Display the plot    
fig

<a href="#top">Top</a>

---

## 9. Draw Topography

Spatial envelopes are required for topo requests, which can become slow to download and render for large (CONUS) maps.

In [ ]:
# Create new request for l
request = DataAccessLayer.newDataRequest("topo")
request.addIdentifier("group", "/")
request.addIdentifier("dataset", "full")
request.setEnvelope(envelope)
gridData = DataAccessLayer.getGridData(request)
grid=gridData[0]
topo=ma.masked_invalid(grid.getRawData()) 
lons, lats = grid.getLatLonCoords()
# print(topo.min()) # minimum elevation in our domain (meters)
# print(topo.max()) # maximum elevation in our domain (meters)

# Plot topography
cs = ax.contourf(lons, lats, topo, 80, cmap=plt.get_cmap('terrain'),alpha=0.1, extend='both')
cbar = fig.colorbar(cs, shrink=0.5, orientation='horizontal')
cbar.set_label("topography height in meters")

# Display the plot
fig

<a href="#top">Top</a>

---

## See also

Documentation for:

* [AWIPS Maps Database Reference List](http://unidata.github.io/awips2/python/maps-database/#mapdatacwa)
* [awips.DataAccessLayer](http://unidata.github.io/python-awips/api/DataAccessLayer.html)
* [awips.PyGeometryData](http://unidata.github.io/python-awips/api/PyGeometryData.html)
* [matplotlib.pyplot](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.html)
* [matplotlib.pyplot.subplot](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.subplot.html)
* [matplotlib.pyplot.contourf](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.contourf.html)


### Related Notebooks:

* [python-awips: How to Access Data](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/dataAccess/python-awips-HowToAccessData.ipynb)


<a href="#top">Top</a>